In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
lemmatizer = WordNetLemmatizer()

In [28]:
intents = json.loads(open('/content/intents.json').read())

In [29]:
words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

In [30]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [32]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

In [34]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [35]:
training = []
outputEmpty = [0] * len(classes)

In [36]:
for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

In [37]:
random.shuffle(training)
training = np.array(training)

In [39]:
trainX = training[:, :len(words)]
trainY = training[:, len(words):]

In [40]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

In [41]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print('Done')

Epoch 1/200
5/5 [==============================] - 1s 5ms/step - loss: 1.5945 - accuracy: 0.1200
Epoch 2/200
5/5 [==============================] - 0s 3ms/step - loss: 1.5423 - accuracy: 0.4800
Epoch 3/200
5/5 [==============================] - 0s 4ms/step - loss: 1.5992 - accuracy: 0.2000
Epoch 4/200
5/5 [==============================] - 0s 3ms/step - loss: 1.5292 - accuracy: 0.3200
Epoch 5/200
5/5 [==============================] - 0s 4ms/step - loss: 1.5530 - accuracy: 0.3200
Epoch 6/200
5/5 [==============================] - 0s 4ms/step - loss: 1.4906 - accuracy: 0.2400
Epoch 7/200
5/5 [==============================] - 0s 3ms/step - loss: 1.4457 - accuracy: 0.4000
Epoch 8/200
5/5 [==============================] - 0s 4ms/step - loss: 1.3840 - accuracy: 0.4400
Epoch 9/200
5/5 [==============================] - 0s 3ms/step - loss: 1.3759 - accuracy: 0.4400
Epoch 10/200
5/5 [==============================] - 0s 4ms/step - loss: 1.2399 - accuracy: 0.5600
Epoch 11/200
5/5 [===========

In [43]:
from keras.models import load_model

In [44]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('/content/intents.json').read())

In [45]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [46]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [47]:
def bag_of_words (sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [48]:
def predict_class (sentence):
    bow = bag_of_words (sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes [r[0]], 'probability': str(r[1])})
    return return_list

In [49]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice (i['responses'])
            break
    return result

In [50]:
print("GO! Bot is running!")

GO! Bot is running!


In [52]:
! pip install wikipedia-api

In [53]:
import wikipediaapi

In [56]:
def search_wikipedia(query):
    # Create a Wikipedia API object
    wiki_wiki = wikipediaapi.Wikipedia('en',
                                       user_agent='/content/chatbot_model.h5 (subhadeep.dey_ds24sp@praxistech.school)') # Replace with your app name and contact info)

    # Fetch the page for the query
    page = wiki_wiki.page(query)

    # Check if the page exists
    if page.exists():
        # Get the summary of the page
        summary = page.summary
        response = f"Here is what I found on Wikipedia: {summary}"
    else:
        response = "Sorry, I couldn't find any information on Wikipedia about that."

    return response

In [57]:
# Example usage
user_query = "Albert Einstein"
response = search_wikipedia(user_query)
print(response)

TypeError: Wikipedia.__init__() got multiple values for argument 'user_agent'

In [51]:
while True:
    message = input("")
    ints = predict_class (message)
    res = get_response (ints, intents)
    print (res)

hello
1/1 [==============================] - 0s 65ms/step
Hello
wikipedia
1/1 [==============================] - 0s 35ms/step
Hello


KeyboardInterrupt: Interrupted by user